In [ ]:
%load_ext autoreload

In [ ]:
%autoreload
import pandas as pd
from utils import (
    df_to_excel_col_dict,
    getSharesIdx,
    getSharesIdxCI95,
    load_tnc_trips,
    out_dir,
    prep_data_1d,
    prep_data_2d,
    tour_extract_wkday_dir,
    write_to_excel,
)
from xlsxwriter.utility import xl_rowcol_to_cell


In [ ]:
out_filepath = out_dir / "04a_TNCTrips_NumPax.xlsx"

In [ ]:
# column tnc_pooled no longer available in 2022
# in 2019: tnc_occupancy = tnc_pooled + num_travelers (see 2019 ipynb for exact logic)
# in 2022: we just use num_travelers directly
trip = load_tnc_trips(tour_extract_wkday_dir)

# dpurp2 is the dpurp with dpurp == 0 (home) replaced by opurp
trip["dpurp2"] = trip["dpurp"]
trip.loc[trip["dpurp"] == 0, "dpurp2"] = trip.loc[trip["dpurp"] == 0, "opurp"]


In [ ]:
df_to_excel_col_dict |= {
    "dpurp2": {
        "desc": "dpurp (= opurp, if dpurp==home)",
        "col": "dpurp2",
        "vals": range(1, 8),
        "labels": [
            "1_Work",
            "2_School",
            "3_Escort",
            "4_PersBus",
            "5_Shop",
            "6_Meal",
            "7_SocRec",
        ],
    }
}


In [ ]:
writer = pd.ExcelWriter(out_filepath, engine="xlsxwriter")
workbook = writer.book
format1 = workbook.add_format({"num_format": "#,##0.0"})

In [ ]:
row = 0
sname = "Weekday"
d1_dict = df_to_excel_col_dict["num_travelers_cat"]

title = "TNC Trips by " + d1_dict["desc"]
tab = prep_data_1d(
    trip,
    d1_dict["desc"],
    d1_dict["col"],
    "trexpfac",
    d1_dict["vals"],
    d1_dict["labels"],
)
row = write_to_excel(writer, tab, sname, title, row)

title = "Column Shares by " + d1_dict["desc"]
row = write_to_excel(writer, getSharesIdx(tab.copy()), sname, title, row)

tab2 = tab.copy()
tab2.iloc[-1, -1] = tab2.iloc[-1, 0]

title = "Column Shares 95% CI by " + d1_dict["desc"]
row = write_to_excel(writer, getSharesIdxCI95(tab.copy()), sname, title, row)

tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
_ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [ ]:
wt_cols = ["count", "trexpfac"]
wt_desc = ["(Unweighted)", "(Weighted)"]

for key in ["dpurp2", "tnc_type"]:
    d2_dict = df_to_excel_col_dict[key]
    row = 0
    sname = d2_dict["desc"]

    for wc, wd in zip(wt_cols, wt_desc):
        title = "TNC Trips by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
        tab = prep_data_2d(
            trip,
            d1_dict["col"],
            d1_dict["vals"],
            d1_dict["labels"],
            d2_dict["col"],
            d2_dict["vals"],
            d2_dict["labels"],
            wc,
        )
        row = write_to_excel(writer, tab.astype("float64"), sname, title, row)

        if wc == "count":
            tab2 = tab.copy()
        else:
            tab2.iloc[:-1, :-1] = tab.iloc[:-1, :-1]

        title = (
            "Column Shares by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
        )
        row = write_to_excel(writer, getSharesIdx(tab.copy()), sname, title, row)

        title = (
            "Column Shares 95% CI by "
            + d2_dict["desc"]
            + " and "
            + d1_dict["desc"]
            + " "
            + wd
        )
        row = write_to_excel(writer, getSharesIdxCI95(tab2.copy()), sname, title, row)
    tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
    _ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [ ]:
writer.close()